<a href="https://colab.research.google.com/github/hushuangwei/rgn2-replica/blob/main/rgn2_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* How would coevolution implicitly affect the language model training?
* Would kNN-style unsupervised learning useful for RGN2?

In [1]:
!git clone https://github.com/hushuangwei/rgn2-replica.git

Cloning into 'rgn2-replica'...
remote: Enumerating objects: 950, done.
remote: Counting objects: 100% (950/950), done.
remote: Compressing objects: 100% (622/622), done.
remote: Total 950 (delta 633), reused 609 (delta 317), pack-reused 0
Receiving objects: 100% (950/950), 15.80 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (633/633), done.


In [2]:
# https://blog.csdn.net/NEUdeep/article/details/115724826
!export PYTHONWARNINGS='ignore:semaphore_tracker:UserWarning'

In [ ]:
!pip install wandb sidechainnet einops proDy tqdm datasets transformers x-transformers pytorch-lightning fair-esm En-transformer pytorch3d invariant_point_attention

In [4]:
%cd rgn2-replica/scripts/

/content/rgn2-replica/scripts


One can skip this google drive setting and download sidechainnet data directly but more slowly

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!mkdir -p /content/rgn2-replica/scripts/sidechainnet_data/
!cp /content/drive/MyDrive/protein/sidechainnet_casp12_90.pkl /content/rgn2-replica/scripts/sidechainnet_data/

In [7]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [29]:
!nohup python train_rgn2.py --device cuda:0 --wb_entity hushuangwei \
   --wb_proj rgn2_replica --run_name RGN2_ipa_1e-4 \
   --min_len_valid 50 --casp_version 12 --scn_thinning 90 \
   --min_len 0 --max_len 384 --input_dropout 0.1 --num_layers 6 \
   --bidirectional 1 --layer_type LSTM --act aconc --num_recycles_train 8 \
   --refiner_args "{\"refiner_type\": \"IPA\"}" \
   > RGN2X_vanillaLSTM_full_run_logs.txt 2>&1 &

In [ ]:
!tail -f RGN2X_vanillaLSTM_full_run_logs.txt

wandb: Currently logged in as: hushuangwei (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.12.4
wandb: Syncing run RGN2_ipa_1e-4
wandb: ⭐️ View project at https://wandb.ai/hushuangwei/rgn2_replica
wandb: 🚀 View run at https://wandb.ai/hushuangwei/rgn2_replica/runs/372ib897
wandb: Run data is saved locally in /content/rgn2-replica/scripts/wandb/run-20211010_110523-372ib897
wandb: Run `wandb offline` to turn off syncing.
tcmalloc: large alloc 2609438720 bytes == 0x55887bc14000 @  0x7f57e6e5eb6b 0x7f57e6e7e379 0x7f57e0a7526e 0x7f57e0a769e2 0x7f56ff8becb9 0x7f57a1daa759 0x5586097c1045 0x558609781c52 0x5586097f4c25 0x5586097efced 0x5586097827f3 0x5586097822f9 0x5586098c935d 0x558609838a0b 0x5586097813a1 0x558609872e1d 0x5586097f4e99 0x5586097efced 0x5586096c1e2b 0x5586097f1fe4 0x5586097ef9ee 0x558609782bda 0x5586097f1737 0x5586097ef9ee 0x558609782bda 0x5586097f1737 0x558609782afa 0x5586097f0915 0x558609782afa 0x5586097f0915 0x558609782afa
tcmalloc: la

In [27]:
!pkill -f train_rgn2.py

In [28]:
!ps aux | grep train_rgn2.py

root        1088  0.0  0.0  39200  6552 ?        S    09:57   0:00 /bin/bash -c ps aux | grep train_rgn2.py
root        1090  0.0  0.0  38572  5608 ?        S    09:57   0:00 grep train_rgn2.py


In [53]:
!nvidia-smi

Tue Oct  5 08:13:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0   103W / 300W |  11293MiB / 16160MiB |     59%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# below is unfinished

In [ ]:
def build_visualizable_structures(model, data, mode=None):
  """Build visualizable structures for one batch of model's predictions on data."""
  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  with torch.no_grad():
    for batch in data:
      if mode == "seqs":
        model_input = batch.seqs.to(device)
      elif mode == "pssms":
        model_input = batch.seq_evo_sec.to(device)

      # Make predictions for angles, and construct 3D atomic coordinates
      predicted_angles_sincos = model(model_input)
      predicted_angles = inverse_trig_transform(predicted_angles_sincos)
      sb = scn.BatchedStructureBuilder(batch.int_seqs, predicted_angles.cpu())
      break
  return sb

In [ ]:
import sidechainnet as scn
d = scn.load(casp_version=12,
             thinning=90,
             with_pytorch="dataloaders",
             aggregate_model_input=False, 
             seq_as_onehot=False, 
             batch_size=8, 
             dynamic_batching=True)

SidechainNet was loaded from ./sidechainnet_data/sidechainnet_casp12_90.pkl.


In [ ]:
import torch
seqonly_model = "/content/rgn2-replica/scripts/rgn2_models/RGN2X_vanillaLSTM_full_run@_128K.pt"
# 
# seqonly_model =  # todo: read the model
s = build_visualizable_structures(seqonly_model, d["train"], mode="seqs")
s.to_3Dmol(2)